In [1]:
import numpy as np
import pandas as pd
# import nltk
# import re

# import gspread
# from gspread.exceptions import WorksheetNotFound
# from gspread_dataframe import get_as_dataframe, set_with_dataframe


from Levenshtein import ratio
from fuzzy import Soundex
# import scipy.sparse as sps
# import scipy.cluster.hierarchy as sch

from collections import Counter

# import spacy
# from spacy.lemmatizer import Lemmatizer
# from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Whisker, Plot
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, \
BoxZoomTool, WheelZoomTool, ResetTool, PanTool, ColumnDataSource, LabelSet, FactorRange
from bokeh.models.graphs import from_networkx

from bokeh.layouts import gridplot



# from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, \
# BoxZoomTool, WheelZoomTool, ResetTool, PanTool, ColumnDataSource, LabelSet

# from bokeh.plotting import figure
# from bokeh.transform import factor_cmap
from bokeh.palettes import Category20_20, Accent

# from bokeh.layouts import row

# from sklearn.feature_extraction.text import CountVectorizer

import networkx as nx

# from bokeh.models.graphs import from_networkx

import random
import itertools
import math

import pprint
pp = pprint.PrettyPrinter(indent=4)




# from iteration_utilities import flatten

# import matplotlib as mpl
# import matplotlib.pyplot as plt

In [2]:
depositions_df = pd.read_json('../converted_json/all_depositions.json')


In [3]:
depositions_df.iloc[4186]

country                                                           Ireland
creation_date                                                   1643-3-10
deponent_county                                                    [Cork]
deponent_town                                             [Michellstowne]
deposition_type                                                   [Bysse]
filename                                           dep_824148r131_tei.xml
nature_of_deposition                   [arson, Multiple Killing, Robbery]
participants_number                                                     0
people_list                                                            []
signed_by               [{'role': 'Commissioner', 'name': 'Philip Byss...
title                                                                 NaN
Name: 4186, dtype: object

In [ ]:
depositions_df['creation_date_period'] = pd.PeriodIndex(depositions_df['creation_date'], freq='d')

In [ ]:
depositions_df.dtypes

In [9]:
type(depositions_df['deponent_town'][2])

list

In [ ]:
depositions_df['creation_year'] = depositions_df['creation_date'].str.slice(start=0, stop=4)

In [ ]:
depositions_df['creation_year'] = depositions_df['creation_year'].fillna('Unknown')

In [ ]:
group_a = depositions_df.groupby('creation_year')

In [ ]:
group_a.describe()

# Number of depositions per year

In [ ]:
output_notebook()

p = figure(plot_width=800, plot_height=600, x_range=group_a, title="Depositions by Year",
           toolbar_location=None, tools="")

p.vbar(x='creation_year', top='participants_number_count', width=0.8, source=group_a)

p.y_range.start = 0
p.xgrid.grid_line_color = None

show(p)

# Number of participants per deposition

In [ ]:
depositions_df['participants_number'].describe()

In [ ]:
depositions_df[depositions_df['participants_number'] == 0]

In [ ]:
part_hist, edges = np.histogram(depositions_df['participants_number'], bins = list(range(0,50,1)))
participants = pd.DataFrame({'part_hist' : part_hist, 'left': edges[:-1], 'right': edges[1:]})   

In [ ]:
participants

In [ ]:
output_notebook()

p = figure(plot_width=800, 
           plot_height=600, 
           title="Histogram of Participants (up to 100) in 1641 Depositions",
           x_axis_label = "Number of participants",
           y_axis_label = "Number of depositions")

p.quad(bottom=0,
       top=participants['part_hist'],
       left=participants['left'],
       right=participants['right'],
       fill_color='red',
       line_color='black')

show(p)

# People's names

In [ ]:
name_counter = Counter()
people_list = depositions_df['people_list'].tolist()
names_list = []
person2indx = {}
#All participants have only one role (checked)
indx2role = {}
for i, dep_part in enumerate(people_list):
    names = []
    for person in dep_part:
        person_str = ''
        if 'forename' in person:
            person_str += person['forename']
        else:
            person_str += 'None'
        person_str += ' '
        
        if 'surname' in person:
            person_str += person['surname']
        else:
            person_str += 'None'
    
        if person_str not in person2indx:
            idx = len(person2indx)
            person2indx[person_str] = idx
            indx2role[idx] = {i : person['role']}
            if person_str == 'Dauid Roch':
                print('1',person_str, person['role'], i, idx)
        else:
            idx = person2indx[person_str]
            indx2role[idx][i] = person['role']
            if person_str == 'Dauid Roch':
                print('2',person_str, person['role'], i)
        name_counter[person_str] += 1
        names.append(person_str)
    names_list.append(names)

indx2person = {v:k for k,v in person2indx.items()}

In [ ]:
for i,name in enumerate(name_counter.most_common(1000)):
    if name[0].split(' ')[0] == '*':
        print(i, name)

In [ ]:
sorted(list(filter(lambda x: x.split(' ')[0] == '*', name_counter.keys())))

In [ ]:
sorted(list(filter(lambda x: x.split(' ')[1] == 'Roch', name_counter.keys())))

# People network

In [ ]:
soundex = Soundex(4)

In [ ]:
G = nx.Graph()
for i, dep_names in enumerate(names_list):
    for person_a, person_b in itertools.combinations(dep_names, 2):
        G.add_edge(person2indx[person_a], person2indx[person_b], deposition=i)
    

In [ ]:
print(names_list[783])
print(people_list[4010])

In [ ]:
indx2role

In [ ]:
role_counter = Counter([a for d in indx2role.values() for a in d.values()])

role2color = {n[0]:Category20_20[i] for i,n in enumerate(role_counter.most_common(20))}

In [ ]:
role2color

In [ ]:
for node in G.nodes():
    G.nodes[node]['full_name'] = indx2person[node]

In [ ]:
adams_indxs = {k:v for k,v in person2indx.items() if k.split(" ")[1] == "Adams"}


In [ ]:
adams_indxs

In [ ]:
def merge_nodes(G,nodes, new_node, attr_dict=None):
    """
    Merges the selected `nodes` of the graph G into one `new_node`,
    meaning that all the edges that pointed to or from one of these
    `nodes` will point to or from the `new_node`.
    attr_dict and **attr are defined as in `G.add_node`.
    """
    
    G.add_node(new_node, **attr_dict) # Add the 'merged' node
    edges_to_add = []
    for n1,n2,data in G.edges(data=True):
        # For all edges related to one of the nodes to merge,
        # make an edge going to or coming from the `new gene`.
        if n1 in nodes:
            edges_to_add.append((new_node,n2,data))
        elif n2 in nodes:
            edges_to_add.append((n1,new_node,data))
    
    G.add_edges_from(edges_to_add)
    
    for n in nodes: # remove the merged nodes
        G.remove_node(n)

In [ ]:
def get_subgraph_for_pair(name_1, name_2, G):
    name_1_indx = person2indx[name_1]
    name_2_indx = person2indx[name_2]
    pair = [name_1_indx, name_2_indx]
    nodes = []
    graph_1 = nx.ego_graph(G, name_1_indx)
    graph_2 = nx.ego_graph(G, name_2_indx)
    merge_graph = nx.compose(graph_1, graph_2)
    
    remove_edges = [] 
    for edge in merge_graph.edges():
        if name_1_indx not in edge and name_2_indx not in edge:
            remove_edges.append(edge)
    merge_graph.remove_edges_from(remove_edges)
    
    depositions = set([edge[2]['deposition'] for edge in merge_graph.edges(data=True)])
    
    for node in merge_graph.nodes(data=True):
        print(node)
        print(indx2role[node[0]].keys())
        print(depositions)
        role_key = list(depositions.intersection(set(indx2role[node[0]].keys())))[0]
        node_role = indx2role[node[0]][role_key]
            
        merge_graph.nodes[node[0]]['role'] = node_role
        merge_graph.nodes[node[0]]['role_color'] = role2color[node_role] if node_role in role2color else "black"
        
        if node[0] == name_1_indx or node[0] == name_2_indx:
            merge_graph.nodes[node[0]]['size'] = 0.07
        else:
            merge_graph.nodes[node[0]]['size'] = 0.035

    
    
    print(len(depositions))
    deposition_colors = {}
    scale_i = 0
    for edge in merge_graph.edges(data=True):
        if edge[2]['deposition'] not in deposition_colors:
            deposition_colors[edge[2]['deposition']] = Accent[3][scale_i]
            scale_i += 1
        edge[2]['deposition_color'] = deposition_colors[edge[2]['deposition']]
        edge[2]['line_alpha'] = 1
        
    node_comb = itertools.combinations(merge_graph.nodes, 2)
#     invisible_edges = []
#     for idx_a, idx_b in node_comb:
#         sim = ratio(indx2person[idx_a], indx2person[idx_b])
#         if sim > 0.8:
#             print(indx2person[idx_a], '/', indx2person[idx_b])
# #             sound_a = " ".join(soundex(k) for k in indx2person[idx_a].split(" "))
# #             sound_b = " ".join(soundex(k) for k in indx2person[idx_b].split(" "))
# #             print(sound_a, sound_b)
# #             print('----------')
#             invisible_edges.append((idx_a, idx_b, {'inv' : True, 
#                                                    'similarity' : sim,
#                                                    'line_alpha': 0,
#                                                    'weight': 5 + sim}))
                                   
#     merge_graph.add_edges_from(invisible_edges)
    
    merge_pairs = {}
    for idx_a, idx_b in node_comb:
        sim = ratio(indx2person[idx_a], indx2person[idx_b])
        if sim > 0.8:
            attrs = {
                'full_name' : merge_graph.nodes[idx_a]['full_name'],
                'role' : merge_graph.nodes[idx_a]['role'],
                'role_color' : merge_graph.nodes[idx_a]['role_color'],
                'size' : merge_graph.nodes[idx_a]['size']
                
            }
            merge_pairs[(idx_a, idx_b)] = attrs
    
    for (idx_a, idx_b), attrs in merge_pairs.items():
        merge_nodes(merge_graph, [idx_a, idx_b], idx_a+idx_b, attrs)
    

    return merge_graph

# def get_subgraph_for_pair(name_1, name_2, G):
#     print('Hello')
#     name_1_indx = person2indx[name_1]
#     name_2_indx = person2indx[name_2]
#     ego_g = nx.ego_graph(G, name_1_indx)
#     return ego_g
    
    

In [ ]:
interactions_c_a = Counter()
for adam_name, indx in adams_indxs.items():
    if 'Anselmus' in adam_name:
        for neighbor in G.neighbors(indx):
            interactions_c_a[neighbor] += 1
# interactions_c.most_common()
print(sorted(list(interactions_c_a.keys())))

In [ ]:
# interactions_c_b = Counter()
# for adam_name, indx in adams_indxs.items():
#     if 'Ancelmus' in adam_name:
#         for neighbor in G.neighbors(indx):
#             interactions_c_b[neighbor] += 1
# print(sorted(list(interactions_c_b.keys())))

In [ ]:
# interactions_int = set(list(interactions_c_a.keys())).intersection(set(list(interactions_c_b.keys())))
# for el in interactions_int:
#     print(indx2person[el])

In [ ]:
name_pairs = [('Anselmus Adams', 'Ancelmus Adams'), 
              ('Will: Adams', 'William Adams'), 
              ('Giles Adams', 'Gyles Adams'),
              ('Randle Adams', 'Randall Adams')]
name_pairs = [('Anselmus Adams', 'Ancelmus Adams'), 
              ('Will: Adams', 'William Adams'), 
              ('Giles Adams', 'Gyles Adams')]
subgraphs = [get_subgraph_for_pair(pair[0], pair[1], G) for pair in name_pairs]

In [ ]:
indx2role[3938]

In [ ]:
subgraphs[0].nodes(data=True)

In [ ]:
indx2role[3938]

In [ ]:
pp.pprint(depositions_df.iloc[4010].people_list)
print('------')
pp.pprint(depositions_df.iloc[4186].people_list)
print('------')
pp.pprint(depositions_df.iloc[4932].people_list)

In [ ]:
output_notebook()
plots = []

for i, G in enumerate(subgraphs[:1]):
    
    #Show network
    plot = Plot(plot_width=800, plot_height=800, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
    plot.title.text = "%s vs %s" % name_pairs[i]
    

  # graph_renderer = from_networkx(G, nx.spring_layout, scale=1, k=0.09, center=(0, 0))
  # graph_renderer = from_networkx(G, nx.kamada_kawai_layout, scale=1.0, dim=2)
    graph_renderer = from_networkx(G, nx.spring_layout, k=.19, center=(0, 0))

    graph_renderer.node_renderer.glyph = Circle(radius="size", fill_color="role_color")
    graph_renderer.edge_renderer.glyph = MultiLine(line_color="deposition_color", 
                                                   line_alpha="line_alpha",
                                                   line_width=2)
    
    node_hover_tool = HoverTool(tooltips=[("full_name", "@full_name"), ("role", "@role")])
#     edge_hover_tool = HoverTool(tooltips=[("deposition", "@deposition")])
#     plot.add_tools(node_hover_tool, edge_hover_tool, BoxZoomTool(), ResetTool(), PanTool(), WheelZoomTool())
    plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool(), PanTool(), WheelZoomTool())
    
    plot.renderers.append(graph_renderer)


    x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
    node_labels = [node[1]['full_name'] for node in G.nodes(data=True)]
    
    source = ColumnDataSource({'x': x, 'y': y,
                             'full_name': node_labels})


    labels = LabelSet(x='x', 
                    y='y', 
                    text='full_name', 
                    source=source,
                    background_fill_color='white',
                    background_fill_alpha=0.8,
                    text_align='center', 
                    text_baseline='middle',
                    x_offset=5,
                    y_offset=15,
                    text_font_size='0.6em')

    plot.renderers.append(labels)
    plots.append(plot)

grid = gridplot(plots, ncols=3)
show(grid)


# Toponyms

In [ ]:
toponyms_list = depositions_df['creation_place'].tolist()
toponyms_counter = Counter()
for i,d in enumerate(toponyms_list):
    for k,v in d.items():
        toponyms_counter[k+':'+v] += 1
print(toponyms_counter.keys())